In [1]:
from pyspark.sql import SparkSession

1. Creamos una SparSession

In [2]:
spark = SparkSession\
    .builder\
        .master("local[*]")\
            .appName('Test')\
                .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


24/04/01 12:32:42 WARN Utils: Your hostname, DESKTOP-SLEQT56 resolves to a loopback address: 127.0.1.1; using 172.25.13.138 instead (on interface eth0)
24/04/01 12:32:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/01 12:32:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1.1 Leemos un archivo .CSV para crear un __DataFrame__

In [3]:
PATH = "/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv"
df = spark.read\
    .option("header", "True") \
        .csv(PATH)

1.2 Aplicamos una __Accion__sobre el __DataFrame__ para mostrar su contenido

In [4]:
df.show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



2. Trabajamos con Esquemas

2.1 Mostramos el esquema de DF anterior __Vemos que todo es string__ y no es lo que queremos.

In [5]:
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- email: string (nullable = true)



2.2 Inferimos el Esquema usando __".option("inferSchema", True)__

In [153]:
df = spark.read.option("Header","True").option("inferSchema", True).csv(PATH)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: integer (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



2.3 Especificar esquema de forma manual.

In [154]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType 

schema_a = StructType ([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("email", StringType(), True),
])

df = spark.read.schema(schema_a).option("header",True).csv(PATH)
df.show(5)
print(df.printSchema())

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)

None


3.1 Usar **Kwargs en Spark

Se usa para no tener tantos __.option()__ anidados remplazandolo por un unico __.options()__

In [26]:
df = spark.read.options(inferSchema=True, Header=True, delimiter=",").csv(PATH)
df.show(2)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 2984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



4. Spark DF from RDDs

Primero creamos un RDD y Luego lo pasamos a DF

In [27]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('Test_rdd').setMaster("local[*]")
spark_rdd = SparkContext.getOrCreate(conf=conf)

In [53]:
PATH = "/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv"

rdd = spark_rdd.textFile(PATH)
header = rdd.first()
rdd_data = rdd.filter(lambda x: x != header)
#print(rdd_data.count())

In [58]:
rdd_data_split = rdd_data.map(lambda x:x.split(","))
rdd_data_f = rdd_data_split.map(lambda x: [int(x[0]), x[1], x[2], x[3], x[4], int(x[5]), x[6]])
rdd_data_f.first()

[28,
 'Female',
 'Hubert Oliveras',
 'DB',
 '02984',
 59,
 'Annika Hoffman_Naoma Fritts@OOP.com']

4.1 Convertimos el __Rdd__ en un __DataFrame__ usando el Header que sacamos el __Rdd__

In [62]:
lista_headers = header.split(",")
df = rdd_data_f.toDF(lista_headers)

In [63]:
df.show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



In [64]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: long (nullable = true)
 |-- email: string (nullable = true)



4.2 Como el esquema no se puede inferir, lo que hacemos es crear nuestro propio esquema, pero en lugar de usar la __transformacion rdd.toDF()__ usamos __spark.createDataFrame(rdd, schema=)__

In [65]:

schema_rdd = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("email", StringType(), True)
])

mi_df = spark.createDataFrame(rdd_data_f, schema=schema_rdd)
print(mi_df.printSchema())

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)

None


5. Seleccion de columnas

5.1 Notación directa, especificando la columna

In [67]:
mi_df.select("age", "gender").show(2)

+---+------+
|age|gender|
+---+------+
| 28|Female|
| 29|Female|
+---+------+
only showing top 2 rows



5.2 Usando __dot notation__

In [68]:
df.select(df.age, df.email).show(2)

+---+--------------------+
|age|               email|
+---+--------------------+
| 28|Annika Hoffman_Na...|
| 29|Margene Moores_Ma...|
+---+--------------------+
only showing top 2 rows



5.3 Usando __col()__

Es una funcion dentro de la libreria __pyspark.sql.functions__

In [70]:
from pyspark.sql.functions import col

df.select(col("age"), col("gender")).show(5)

+---+------+
|age|gender|
+---+------+
| 28|Female|
| 29|Female|
| 28|  Male|
| 29|Female|
| 28|  Male|
+---+------+
only showing top 5 rows



5.4 Mostrar todas las columnas usando __*__

In [72]:
df.select("*").show(4)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 4 rows



5.5 Select de las columnas usando Index

In [76]:
df.select(df.columns[3:]).show(4)

+------+-----+-----+--------------------+
|course| roll|marks|               email|
+------+-----+-----+--------------------+
|    DB|02984|   59|Annika Hoffman_Na...|
| Cloud|12899|   62|Margene Moores_Ma...|
|    PF|21267|   45|Jeannetta Golden_...|
|    DB|32877|   29|Billi Clore_Mitzi...|
+------+-----+-----+--------------------+
only showing top 4 rows



5.6 Podemos combinar todo lo anterior

In [87]:
df.select(col("age"), df.columns[-1]).show(4)

+---+--------------------+
|age|               email|
+---+--------------------+
| 28|Annika Hoffman_Na...|
| 29|Margene Moores_Ma...|
| 28|Jeannetta Golden_...|
| 29|Billi Clore_Mitzi...|
+---+--------------------+
only showing top 4 rows



6. WithColumn()

6.1 Ejemplo de Casteo de una columna

In [95]:
df.withColumn("roll2", col("roll").cast("Integer")).show(5)

+---+------+----------------+------+-----+-----+--------------------+-----+
|age|gender|            name|course| roll|marks|               email|roll2|
+---+------+----------------+------+-----+-----+--------------------+-----+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...| 2984|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|12899|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|21267|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|32877|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|41487|
+---+------+----------------+------+-----+-----+--------------------+-----+
only showing top 5 rows



6.2 Ejemplo de manipulacion de valores

In [98]:
from pyspark.sql.functions import col

df.withColumn("marks2", col("marks") + 5).show(5)

+---+------+----------------+------+-----+-----+--------------------+------+
|age|gender|            name|course| roll|marks|               email|marks2|
+---+------+----------------+------+-----+-----+--------------------+------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|    64|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|    67|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|    50|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|    34|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|    46|
+---+------+----------------+------+-----+-----+--------------------+------+
only showing top 5 rows



6.3 Crear un valor constante

Con .withColumn se puede crear un valor constante, pero hay que __especificarlo con la funcion lit de .sql.functions__

In [100]:
from pyspark.sql.functions import col, lit

df.withColumn('pais',lit('ARG')).show(5)

+---+------+----------------+------+-----+-----+--------------------+----+
|age|gender|            name|course| roll|marks|               email|pais|
+---+------+----------------+------+-----+-----+--------------------+----+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...| ARG|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...| ARG|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...| ARG|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...| ARG|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...| ARG|
+---+------+----------------+------+-----+-----+--------------------+----+
only showing top 5 rows



6.4 Concatenacion de .withColumn

In [103]:
from pyspark.sql.functions import col

df.withColumn('marks', col('marks') - 10).withColumn('marks', col('marks') + 20).show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   69|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   72|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   55|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   39|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   51|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



7. withColumnRename

Lo usamos para cambiar el nombre de una columna y generar un nuevo __DF__

In [11]:
df_rename = df.withColumnRenamed("age", "edad")
df_rename.show(4)

+----+------+----------------+------+-----+-----+--------------------+
|edad|gender|            name|course| roll|marks|               email|
+----+------+----------------+------+-----+-----+--------------------+
|  28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
|  29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
|  28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
|  29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+----+------+----------------+------+-----+-----+--------------------+
only showing top 4 rows



7.1 Si queremos cambiar el nombre de una variable en tiempo de ejecución usamos __.alias()__

In [17]:
from pyspark.sql.functions import col

df_rename.select(col("edad").alias("Age")).show(5)

+---+
|Age|
+---+
| 28|
| 29|
| 28|
| 29|
| 28|
+---+
only showing top 5 rows



8. Filter en __DataFrames__

In [18]:
from pyspark.sql.functions import col

df.filter(col("course")=='DB').show(5)
df.filter(df.course == "DB").show(5)

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 29|  Male|Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|Female| Latia Vanhoose|    DB|122502|   27|Latia Vanhoose_Mi...|
| 29|Female| Latia Vanhoose|    DB|152159|   27|Claude Panos_Sant...|
+---+------+---------------+------+------+-----+--------------------+
only showing top 5 rows

+---+------+---------------+------+------+-----+--------------------+
|age|gender|           name|course|  roll|marks|               email|
+---+------+---------------+------+------+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi

8.2 Condición multiple

In [19]:
df.filter(
            (df.course == "DB") & (df.marks > 50)
        ).show(5)

+---+------+----------------+------+------+-----+--------------------+
|age|gender|            name|course|  roll|marks|               email|
+---+------+----------------+------+------+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB| 02984|   59|Annika Hoffman_Na...|
| 29|  Male| Ernest Rossbach|    DB|111449|   53|Maybell Duguay_Ab...|
| 28|Female|Mickey Cortright|    DB|192537|   62|Ernest Rossbach_M...|
| 28|Female|     Anna Santos|    DB|311589|   79|Celeste Lollis_Mi...|
| 29|  Male|    Paris Hutton|    DB|481229|   57|Clementina Menke_...|
+---+------+----------------+------+------+-----+--------------------+
only showing top 5 rows



8.3 Funcionalidad __isin()__ para condiciones múltiples

In [24]:
from pyspark.sql.functions import col

l_courses = ["DB", "Cloud", "OOP"]
df.filter(col("course").isin(l_courses)).show(2)
df.filter(df.course.isin(l_courses)).show(2)


+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



8.4 Condición __startsWith()_

La usamos para determinar cuando una cadena comienza con una determinada letra co caracter

In [29]:
from pyspark.sql.functions import col

df.filter(df.course.startswith("D")).show(2)
df.filter(col("course").startswith("D")).show(2)

+---+------+---------------+------+-----+-----+--------------------+
|age|gender|           name|course| roll|marks|               email|
+---+------+---------------+------+-----+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+---+------+---------------+------+-----+-----+--------------------+
only showing top 2 rows

+---+------+---------------+------+-----+-----+--------------------+
|age|gender|           name|course| roll|marks|               email|
+---+------+---------------+------+-----+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|   Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
+---+------+---------------+------+-----+-----+--------------------+
only showing top 2 rows



8.5 Filtro con __.contains()__

In [34]:
df.filter(df.name.contains("Eleno")).show(2)
df.filter(col("name").contains("Eleno")).show(2)

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows



8.6 Filtro con __.like()__

In [36]:
from pyspark.sql.functions import col

df.filter(df.name.like("%Ele%")).show(2)
df.filter(col("name").like("%Ele%")).show(2)

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows

+---+------+------------+------+------+-----+--------------------+
|age|gender|        name|course|  roll|marks|               email|
+---+------+------------+------+------+-----+--------------------+
| 29|Female|Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...|
| 28|  Male|Elenore Choy|    PF|422704|   66|Marylee Capasso_N...|
+---+------+------------+------+------+-----+--------------------+
only showing top 2 rows



9. Spark Ejercicio rápido

In [46]:
from pyspark.sql.functions import col, lit
from pyspark.sql .types import StructField, StructType, IntegerType, StringType

In [139]:
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("emails", StringType(), True),
])

In [140]:
df = spark.read.options(header=True, schema=schema).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
df.show(2)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 2 rows



1. Crear una nueva columna __total marks__ con el valor 120


In [49]:
df = df.withColumn("totalMarks", lit(120))

2. Crear una columna __total mark avg__ para cada alumno con la formula 
__(marks/total_marks)*100__

In [50]:
df = df.withColumn("avg", col("marks")/col("totalMarks") * 100)

3. Filtrar los estudiantes con AVG > 80% in __OOP__
4. Filtrar los estudiantes con AVG > 60% in __CLOUD__


In [72]:
df_oop = df.filter(
                    (df.avg>=80) & (df.course == "OOP") 
                )
                 
df_cloud =  df.filter(               
                    (col("avg")>= 60) & (col("course")=="Cloud")
                    )
                


5. Mostrar los nombres y notas de __3__ y __4__


In [77]:
df_oop.select(col("name"), col("marks")).show(2)
df_cloud.select(col("name"), col("marks")).show(2)

+--------------+-----+
|          name|marks|
+--------------+-----+
| Kizzy Brenner|   96|
|Jenna Montague|   98|
+--------------+-----+
only showing top 2 rows

+------------+-----+
|        name|marks|
+------------+-----+
|Claude Panos|   85|
| Billi Clore|   76|
+------------+-----+
only showing top 2 rows



10. Funciones SQL

10.1 Distinct y count

In [83]:
df.select("gender", "age").distinct().show()

+------+---+
|gender|age|
+------+---+
|  Male| 29|
|  Male| 28|
|Female| 28|
|Female| 29|
+------+---+



10.2 dropDuplicates()

In [98]:
df.dropDuplicates(["gender","age"]).show()
df.dropDuplicates(["age"]).show()

+---+------+----------------+------+-----+-----+--------------------+----------+------------------+
|age|gender|            name|course| roll|marks|               email|totalMarks|               avg|
+---+------+----------------+------+-----+-----+--------------------+----------+------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|       120|49.166666666666664|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|       120| 51.66666666666667|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|       120|              37.5|
| 29|  Male|  Cordie Harnois|   OOP|92882|   51|Judie Chipps_Clem...|       120|              42.5|
+---+------+----------------+------+-----+-----+--------------------+----------+------------------+

+---+------+----------------+------+-----+-----+--------------------+----------+------------------+
|age|gender|            name|course| roll|marks|               email|totalMarks|               avg|

11. Spark Ejercicio rápido

In [152]:
from pyspark.sql import SparkSession

schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("emails", StringType(), True),
])

df = spark.read.options(schema=schema, header=True).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: string (nullable = true)
 |-- email: string (nullable = true)



Solucion 1.

In [107]:
df2 = df.select("age", "gender", "course").distinct()
df2.count()

24

Solucion 2.

In [109]:
df.dropDuplicates(["gender", "age", "course"]).count()

24

12. .sort() u .orderBy()

In [119]:
df.sort(col('age').asc(), col('marks').desc()).show(4)

+---+------+--------------+------+-------+-----+--------------------+
|age|gender|          name|course|   roll|marks|               email|
+---+------+--------------+------+-------+-----+--------------------+
| 28|Female|Melani Engberg| Cloud|1872667|   99|Alberta Freund_Ni...|
| 28|  Male|   Niki Klimek|   DSA|5172507|   99|Marylee Capasso_E...|
| 28|Female|  Judie Chipps|   OOP|5451977|   99|Tamera Blakley_Mi...|
| 28|Female|Jalisa Swenson|   MVC|5712033|   99|Dustin Feagins_Hu...|
+---+------+--------------+------+-------+-----+--------------------+
only showing top 4 rows



13. Spark ejercicio rapido - Sort() u orderBy()

In [121]:
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
from pyspark.sql.functions import col

#employee_name,department,state,salary,age,bonus

schema = StructType([
    StructField('employee_name', StringType(), True),
    StructField('department', StringType(), True),
    StructField('state', StringType(), True),
    StructField('salary', IntegerType(), True),
    StructField('age', IntegerType(), True),
    StructField('bonus', IntegerType(), True),
])

df_emp = spark.read.options(schema=schema, header = True).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/OfficeData.csv")
df_emp.count()

9

1. Crear un DF ordernado por la columna __bonus__ de forma ascendente


In [124]:
df_emp.select("*").sort(col("bonus").asc()).show(3)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
+-------------+----------+-----+------+---+-----+
only showing top 3 rows



2. Crear un DF ordenado por las columnas __age__ y __salary__ en orden descendente y ascendente.


In [127]:
df_emp.select("*").sort(col("age").desc(), col("salary").asc()).show(3)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|      Michael|     Sales|   NY| 86000| 56|20000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+
only showing top 3 rows



3. Crear un DF ordenado por __age__ __bonus__ y __salario__ en orden descendente, descentente y ascendente.


In [130]:
df_emp.select("*").sort(col("age").desc(), col("bonus").desc(), col("salary").asc()).show(3)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|      Michael|     Sales|   NY| 86000| 56|20000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+
only showing top 3 rows



14. GroupBy en DF

14.1 Unica columna

In [12]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType


schemas = StructType([
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("name", StringType(), True),
    StructField("course", StringType(), True),
    StructField("roll", StringType(), True),
    StructField("marks", IntegerType(), True),
    StructField("emails", StringType(), True),
])

df = spark.read.schema(schemas).option("header","True").csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")

df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- emails: string (nullable = true)



In [18]:
df.groupBy("gender").count().show()
df.groupBy("gender").sum("marks").show()
df.groupBy("gender").avg("marks").show()

+------+-----+
|gender|count|
+------+-----+
|Female|  501|
|  Male|  499|
+------+-----+

+------+----------+
|gender|sum(marks)|
+------+----------+
|Female|     29636|
|  Male|     30461|
+------+----------+

+------+------------------+
|gender|        avg(marks)|
+------+------------------+
|Female|59.153692614770456|
|  Male| 61.04408817635271|
+------+------------------+



14.2 groupBy en mas de una columna y agregaciones multiples

In [32]:
from pyspark.sql.functions import col
from pyspark.sql.functions import sum, avg, max, min, mean, count


df.groupBy(col("gender"), col("course"))\
    .agg(count("*"),\
        min("marks"), \
            sum(col("marks"))).show(4)

+------+------+--------+----------+----------+
|gender|course|count(1)|min(marks)|sum(marks)|
+------+------+--------+----------+----------+
|Female| Cloud|     106|        20|      6316|
|Female|   OOP|      82|        21|      4682|
|  Male|    PF|      97|        20|      5960|
|  Male|    DB|      82|        20|      5073|
+------+------+--------+----------+----------+
only showing top 4 rows



14.3 groupBy  + Alias()

In [28]:
df.groupBy(col("gender"), col("age"))\
    .agg(count("*").alias("cantidad_total")\
        ,min("marks").alias("nota_minima")\
            , sum(col("marks")).alias("suma_notas")).show()

+------+---+--------------+-----------+----------+
|gender|age|cantidad_total|nota_minima|suma_notas|
+------+---+--------------+-----------+----------+
|Female| 29|           245|         20|     14419|
|Female| 28|           256|         20|     15217|
|  Male| 28|           238|         20|     14664|
|  Male| 29|           261|         20|     15797|
+------+---+--------------+-----------+----------+



15. GroupBy() y Filtering

Vemos que el agrupado siempre se hace despues del __.filter()__ con los registros resultantes

In [35]:
df.filter(col("marks") > 35).groupBy(col("gender")).agg(count("*")).show()
df.filter(col("marks") <= 35).groupBy(col("gender")).agg(count("*")).show()
df.groupBy(col("gender")).agg(count("*")).show()



+------+--------+
|gender|count(1)|
+------+--------+
|Female|     399|
|  Male|     410|
+------+--------+

+------+--------+
|gender|count(1)|
+------+--------+
|Female|     102|
|  Male|      89|
+------+--------+

+------+--------+
|gender|count(1)|
+------+--------+
|Female|     501|
|  Male|     499|
+------+--------+



15.1 Concatenacion y agrupados

In [47]:
df_aux = df.filter(col("marks") > 35).groupBy(col("gender"),col("age")).agg(count("*").alias("total_gender"))
df_aux.filter(df_aux.total_gender > 50).show()

+------+---+------------+
|gender|age|total_gender|
+------+---+------------+
|Female| 29|         194|
|Female| 28|         205|
|  Male| 28|         197|
|  Male| 29|         213|
+------+---+------------+



In [48]:
df.filter(col("marks") > 35).groupBy(col("gender"),col("age")).agg(count("*").alias("total_gender")).filter(col("total_gender") > 50).show()

+------+---+------------+
|gender|age|total_gender|
+------+---+------------+
|Female| 29|         194|
|Female| 28|         205|
|  Male| 28|         197|
|  Male| 29|         213|
+------+---+------------+



16. Spark ejercicio rápido GroupBy()

In [22]:
from pyspark.sql.functions import col, max, min, avg, count, sum
from pyspark.sql.types import StructField, StructType, IntegerType, StringType

#age,gender,name,course,roll,marks,email

schema = StructType([
    StructField("age",IntegerType(), True),
    StructField("gender",StringType(), True),
    StructField("name",StringType(), True),
    StructField("course",StringType(), True),
    StructField("roll",StringType(), True),
    StructField("marks",IntegerType(), True),
    StructField("email",StringType(), True)
    
])

df = spark.read.schema(schema).option("header","True").csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



In [20]:
df.show(5)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



1. El numero total de estudiantes en cada curso.


In [37]:
df.groupBy(col("course"))\
    .agg(count('*')\
        .alias('Total_alumnos'))\
            .sort(col("Total_alumnos").desc())\
                .show(2)

+------+-------------+
|course|Total_alumnos|
+------+-------------+
| Cloud|          192|
|   DSA|          176|
+------+-------------+
only showing top 2 rows



2. EL numero total de H y M en cada curso.


In [36]:
df.groupBy(col("course"), col("gender"))\
    .agg(count("*")\
        .alias("total"))\
            .sort(col("total").asc())\
                .show(4)

+------+------+-----+
|course|gender|total|
+------+------+-----+
|    PF|Female|   69|
|   OOP|  Male|   70|
|   MVC|Female|   71|
|    DB|Female|   75|
+------+------+-----+
only showing top 4 rows



3. EL total de notas por cada genero en cada curso.


In [35]:
df.groupBy(col("course"), col("gender")).agg(sum(col("marks")).alias("Suma_total")).show(4)

+------+------+----------+
|course|gender|Suma_total|
+------+------+----------+
|   OOP|  Male|      4234|
|    DB|  Male|      5073|
| Cloud|Female|      6316|
|   MVC|  Male|      5241|
+------+------+----------+
only showing top 4 rows



4. Por cada curso, el maximo, minimo y AVG por grupo de edad.


In [38]:
df.groupBy(col("course"))\
    .agg(avg(col("age")).alias("avg_age")\
        , max(col("age")).alias("max_age") \
            , min(col("age")).alias("min_age"))\
                .show(2)

+------+------------------+-------+-------+
|course|           avg_age|max_age|min_age|
+------+------------------+-------+-------+
|    PF| 28.52409638554217|     29|     28|
|    DB|28.477707006369428|     29|     28|
+------+------------------+-------+-------+
only showing top 2 rows



17. Spark ejercicio rápido WordCount

In [10]:
from pyspark.sql.functions import avg, sum, min, max, count, col
from pyspark.sql.types import StructField, StructType

header = ['Word']

schema = StructType([
    StructField("Word", StringType(), True)
])

df_words = spark.read.option("header", False).schema(schema)\
    .csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/WordData.txt")
df_words.show(3)

+-----+
| Word|
+-----+
|Apple|
|  Mic|
|  Mic|
+-----+
only showing top 3 rows



In [52]:
df_words.groupBy('Word').agg(count("*").alias("Total")).sort(col("Total")).show()

+------+-----+
|  Word|Total|
+------+-----+
|  Book|    5|
|Mobile|    5|
|Laptop|    5|
|   Bag|    5|
|   Mic|   10|
| Apple|   10|
| Chair|   15|
+------+-----+



19. UDF

Creamos una funcion que devuelva en una nueva columna la suma entre __salario__ y __bonus__

In [61]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, lit


df = spark.read.option("inferSchema", True).option("header", True).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/OfficeData.csv")
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- bonus: integer (nullable = true)



In [63]:
def get_total_slary(salary, bonus):
    return salary + bonus

getTotalSalaryUDF = udf(lambda x,y: get_total_slary(x,y), IntegerType())

df.withColumn("total_salario", getTotalSalaryUDF(col("salary"), col("bonus")) ).show(10)

+-------------+----------+-----+------+---+-----+-------------+
|employee_name|department|state|salary|age|bonus|total_salario|
+-------------+----------+-----+------+---+-----+-------------+
|        James|     Sales|   NY| 90000| 34|10000|       100000|
|      Michael|     Sales|   NY| 86000| 56|20000|       106000|
|       Robert|     Sales|   CA| 81000| 30|23000|       104000|
|        Maria|   Finance|   CA| 90000| 24|23000|       113000|
|        Raman|   Finance|   CA| 99000| 40|24000|       123000|
|        Scott|   Finance|   NY| 83000| 36|19000|       102000|
|          Jen|   Finance|   NY| 79000| 53|15000|        94000|
|         Jeff| Marketing|   CA| 80000| 25|18000|        98000|
|        Kumar| Marketing|   NY| 91000| 50|21000|       112000|
+-------------+----------+-----+------+---+-----+-------------+



20. Spark Ejercicio rapido - UDF .withColumn()

In [18]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, FloatType

df = spark.read.option("inferSchema", True).option("header", True).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/OfficeData.csv")
df.printSchema()

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- bonus: integer (nullable = true)




1. Crear una nueva columna que muestre el incremento en el sueldo segun:
- Si el empleado esta en NY, 10% en salario y 5% en bonus
- Si esta en CA, 12% en salario y 3% en bonus

In [21]:
def increment_salary(city, salary, bonus):
    suma = 0
    if city == "NY":
        suma = salary * 1.15 + bonus * 1.05
    elif city == "CA":
        suma = salary * 1.12 + bonus *1.03
        
    return suma


get_salary = udf(lambda x,y,z: increment_salary(x,y,z), FloatType())

df.withColumn("incremento_salary", get_salary(col("state"), col("salary"), col("bonus"))).show()

+-------------+----------+-----+------+---+-----+-----------------+
|employee_name|department|state|salary|age|bonus|incremento_salary|
+-------------+----------+-----+------+---+-----+-----------------+
|        James|     Sales|   NY| 90000| 34|10000|         114000.0|
|      Michael|     Sales|   NY| 86000| 56|20000|         119900.0|
|       Robert|     Sales|   CA| 81000| 30|23000|         114410.0|
|        Maria|   Finance|   CA| 90000| 24|23000|         124490.0|
|        Raman|   Finance|   CA| 99000| 40|24000|         135600.0|
|        Scott|   Finance|   NY| 83000| 36|19000|         115400.0|
|          Jen|   Finance|   NY| 79000| 53|15000|         106600.0|
|         Jeff| Marketing|   CA| 80000| 25|18000|         108140.0|
|        Kumar| Marketing|   NY| 91000| 50|21000|         126700.0|
+-------------+----------+-----+------+---+-----+-----------------+



21. Cache y Persist

In [12]:
from pyspark.sql.types import IntegerType, StructField, StringType, StructType
from pyspark.sql.functions import col

In [16]:
schema_df = StructType([
    StructField("age", IntegerType(), False),
    StructField("gender", StringType(), False),
    StructField("name", StringType(), False),
    StructField("course", StringType(), False),
    StructField("roll", StringType(), False),
    StructField("marks", IntegerType(), False),
    StructField("email", StringType(), False)
])

In [6]:
df = spark.read.option("header","True").schema(schema_df).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
df.show(5)


+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
| 29|Female|    Elenore Choy|    DB|32877|   29|Billi Clore_Mitzi...|
| 28|  Male|  Sheryll Towler|   DSA|41487|   41|Claude Panos_Judi...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- name: string (nullable = true)
 |-- course: string (nullable = true)
 |-- roll: string (nullable = true)
 |-- marks: integer (nullable = true)
 |-- email: string (nullable = true)



* En este ejemplo aplicamos una dos transformaciones, hacemos __cache__ sobre ese __dataframe__ y luego una accion.

In [11]:
df_trx = df.groupBy("gender", "course", "age").count()

In [13]:
df_trx = df_trx.withColumn("dummy", col("age") + 100)

In [14]:
df_trx.cache()

DataFrame[gender: string, course: string, age: int, count: bigint, dummy: int]

In [15]:
df_trx.show(5)

+------+------+---+-----+-----+
|gender|course|age|count|dummy|
+------+------+---+-----+-----+
|Female|   OOP| 29|   39|  129|
|  Male|   MVC| 28|   38|  128|
|Female|   MVC| 28|   34|  128|
|Female|    DB| 28|   40|  128|
|  Male|    PF| 29|   53|  129|
+------+------+---+-----+-----+
only showing top 5 rows



__AGREGAR CONVERTIR A RDD__

22. Spark SQL

In [19]:
mi_df = spark.read.\
    option("header","true").\
        schema(schema_df).\
            csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/StudentData.csv")
            
mi_df.show(3)

+---+------+----------------+------+-----+-----+--------------------+
|age|gender|            name|course| roll|marks|               email|
+---+------+----------------+------+-----+-----+--------------------+
| 28|Female| Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 29|Female|Toshiko Hillyard| Cloud|12899|   62|Margene Moores_Ma...|
| 28|  Male|  Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
+---+------+----------------+------+-----+-----+--------------------+
only showing top 3 rows



In [21]:
mi_df.createOrReplaceTempView("miVista")

df = spark.sql("""
               select gender, age, count(*)
               from miVista
               group by gender, age
               """)

df.show(4)

+------+---+--------+
|gender|age|count(1)|
+------+---+--------+
|Female| 29|     245|
|Female| 28|     256|
|  Male| 28|     238|
|  Male| 29|     261|
+------+---+--------+



23. Write DF

In [22]:
df.write.option("header", "true").csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/agrupados/alumnos.csv")

24. Trabajo integrador.

In [13]:
from pyspark.sql.functions import col, min, max, avg, udf
from pyspark.sql.types import FloatType, IntegerType, StringType, StructType, StructField
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder.\
    master("local").\
        appName("TrabajoIntegrador").\
            getOrCreate()

24/04/01 12:33:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
mi_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("employee_name", StringType(), True),
    StructField("department", StringType(), True),
    StructField("state", StringType(), True),
    StructField("salary", FloatType(), True),
    StructField("age", IntegerType(), True),
    StructField("bonus", IntegerType(), True)
])

In [9]:
mi_df = spark.read.option("header", "true").\
    schema(mi_schema).csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/OfficeDataProject.csv")

mi_df.createOrReplaceTempView("mi_df_tabla")

mi_df.show(4)

+-----------+-----------------+----------+-----+------+---+-----+
|employee_id|    employee_name|department|state|salary|age|bonus|
+-----------+-----------------+----------+-----+------+---+-----+
|       1000|        Nitz Leif| Marketing|   CA|6131.0| 26|  543|
|       1001|  Melissia Dedman|   Finance|   AK|4027.0| 43| 1290|
|       1002|Rudolph Barringer|        HR|   LA|3122.0| 43| 1445|
|       1003|      Tamra Amber|  Accounts|   AK|5717.0| 47| 1291|
+-----------+-----------------+----------+-----+------+---+-----+
only showing top 4 rows



1. Número total de empleados en la compania.

In [10]:
mi_df.count()

1000

In [6]:
spark.sql("""
          select count(*)
          from mi_df_tabla
          """).show()

+--------+
|count(1)|
+--------+
|    1000|
+--------+



2. Número total de departamentos en la compania.

In [7]:
mi_df.select(col("department")).\
    distinct().\
        count()

6

In [8]:
spark.sql("""
          SELECT count(distinct department)
          FROM mi_df_tabla
          """).show()

+--------------------------+
|count(DISTINCT department)|
+--------------------------+
|                         6|
+--------------------------+



3. El nombre de los departamentos en la compania.

Lo podemos hacer usando __distinct()__ o __dropDuplicates()__

In [9]:
mi_df.select(col("department")).\
    distinct().\
        show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



In [10]:
spark.sql("""
          SELECT DISTINCT(department) as nombre_depto
          from mi_df_tabla
          """).show()

+------------+
|nombre_depto|
+------------+
|       Sales|
|          HR|
|     Finance|
|  Purchasing|
|   Marketing|
|    Accounts|
+------------+



4. Número de empleados por cada departamento.

In [11]:
mi_df.groupBy(col("department")).\
    count().\
        show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



In [12]:
spark.sql("""
          SELECT department, count(*)
          FROM mi_df_tabla
          GROUP BY department
          """).show()

+----------+--------+
|department|count(1)|
+----------+--------+
|     Sales|     169|
|        HR|     171|
|   Finance|     162|
|Purchasing|     166|
| Marketing|     170|
|  Accounts|     162|
+----------+--------+



5. Número de empleados en cada estado.

In [24]:
mi_df.groupBy(col("department"), col("state")).count().\
    orderBy(col("department").asc(), col("state").desc()).show()

+----------+-----+-----+
|department|state|count|
+----------+-----+-----+
|  Accounts|   WA|   27|
|  Accounts|   NY|   34|
|  Accounts|   LA|   29|
|  Accounts|   CA|   35|
|  Accounts|   AK|   37|
|   Finance|   WA|   30|
|   Finance|   NY|   31|
|   Finance|   LA|   29|
|   Finance|   CA|   35|
|   Finance|   AK|   37|
|        HR|   WA|   47|
|        HR|   NY|   30|
|        HR|   LA|   41|
|        HR|   CA|   28|
|        HR|   AK|   25|
| Marketing|   WA|   39|
| Marketing|   NY|   30|
| Marketing|   LA|   26|
| Marketing|   CA|   33|
| Marketing|   AK|   42|
+----------+-----+-----+
only showing top 20 rows



In [30]:
spark.sql("""
          SELECT department, state, count(*) as cta
          FROM mi_df_tabla
          GROUP BY department, state
          order by department asc, state desc
          """).show()

+----------+-----+---+
|department|state|cta|
+----------+-----+---+
|  Accounts|   WA| 27|
|  Accounts|   NY| 34|
|  Accounts|   LA| 29|
|  Accounts|   CA| 35|
|  Accounts|   AK| 37|
|   Finance|   WA| 30|
|   Finance|   NY| 31|
|   Finance|   LA| 29|
|   Finance|   CA| 35|
|   Finance|   AK| 37|
|        HR|   WA| 47|
|        HR|   NY| 30|
|        HR|   LA| 41|
|        HR|   CA| 28|
|        HR|   AK| 25|
| Marketing|   WA| 39|
| Marketing|   NY| 30|
| Marketing|   LA| 26|
| Marketing|   CA| 33|
| Marketing|   AK| 42|
+----------+-----+---+
only showing top 20 rows



7. Salario máximo y mínimo en cada departamento en orden ascendente.

In [21]:
mi_df.groupBy(col("department")).agg(min("salary").\
    alias('min_salary'), max("salary").alias("max_salary")).\
        orderBy(col("min_salary").asc()).\
        show()

+----------+----------+----------+
|department|min_salary|max_salary|
+----------+----------+----------+
|   Finance|    1006.0|    9899.0|
|  Accounts|    1007.0|    9890.0|
|        HR|    1013.0|    9982.0|
| Marketing|    1031.0|    9974.0|
|     Sales|    1103.0|    9982.0|
|Purchasing|    1105.0|    9985.0|
+----------+----------+----------+



In [7]:
spark.sql("""
          SELECT department, min(salary) as sal_min, max(salary) as sal_max
          FROM mi_df_tabla
          GROUP BY department
          ORDER BY sal_min desc
          """).show()

+----------+-------+-------+
|department|sal_min|sal_max|
+----------+-------+-------+
|Purchasing| 1105.0| 9985.0|
|     Sales| 1103.0| 9982.0|
| Marketing| 1031.0| 9974.0|
|        HR| 1013.0| 9982.0|
|  Accounts| 1007.0| 9890.0|
|   Finance| 1006.0| 9899.0|
+----------+-------+-------+



8. Nombre de los empleados en NY en __Finance__ cuyo bonus es mayor al bono del promedio de los empleados de NY.

In [17]:
df_bonus = mi_df.filter(col("state") == 'NY').\
    groupBy(col("state")).\
        agg(avg(col("bonus")).
            alias("avg_bonus")).\
                select(col("avg_bonus")).\
                    collect()

__Explicacion__ Lo que hacemos en este caso es:
1. Traer los registros de NY y calcular el promedio.
2. Poner un nombre indentificativo a la columna.
3. Seleccionamos la columna y en lugar de hacer show() hacemos __collect()__

__collect()__ lo que hace es devolver el valor en forma de array donde accedemos por __indice__ y __nombre__

In [19]:
df_bonus[0]['avg_bonus']

1251.3468208092486

In [24]:
mi_df.filter(
                (col("state")== 'NY') 
                & (col('department')=='Finance') 
                & (col('bonus') > df_bonus[0]['avg_bonus'])
            ).\
                select('employee_name').\
                    show()

+--------------------+
|       employee_name|
+--------------------+
|       Vivan Sifford|
|      Herder Gallman|
|          Nena Rocha|
|       Leif Lemaster|
|Ellingsworth Meli...|
|        Escoto Gilma|
|     Georgeanna Laub|
|     Durio Tenenbaum|
|       Juliana Grigg|
|        Tiffani Benz|
|          Nitz Ilana|
|   Phylicia Antonina|
|         Durio Janey|
|       Melissia Jere|
|      Yukiko Kreamer|
|      Nena Kensinger|
|      Antonina Ilana|
+--------------------+



In [11]:
spark.sql("""
          SELECT employee_name
          FROM mi_df_tabla
          WHERE bonus > (
                            SELECT avg(bonus) as avg_bonus
                            FROM mi_df_tabla
                            WHERE state = 'NY'
                            GROUP BY state
                        ) 
            AND  department = 'Finance'
            AND state = 'NY'
              """).show()

+--------------------+
|       employee_name|
+--------------------+
|       Vivan Sifford|
|      Herder Gallman|
|          Nena Rocha|
|       Leif Lemaster|
|Ellingsworth Meli...|
|        Escoto Gilma|
|     Georgeanna Laub|
|     Durio Tenenbaum|
|       Juliana Grigg|
|        Tiffani Benz|
|          Nitz Ilana|
|   Phylicia Antonina|
|         Durio Janey|
|       Melissia Jere|
|      Yukiko Kreamer|
|      Nena Kensinger|
|      Antonina Ilana|
+--------------------+



9. Aumentar $500 el salario de todos los empleados con edad mayor a 45.

In [22]:
def incrementSalary(age, salary):
    if age > 45:
        return salary + 500
    return salary

fdxIncrementSalary = udf(lambda x,y: incrementSalary(x,y), FloatType())

In [30]:
mi_df.\
        withColumn("salary", fdxIncrementSalary(col("age"), col("salary"))).show(4)

+-----------+-----------------+----------+-----+------+---+-----+
|employee_id|    employee_name|department|state|salary|age|bonus|
+-----------+-----------------+----------+-----+------+---+-----+
|       1000|        Nitz Leif| Marketing|   CA|6131.0| 26|  543|
|       1001|  Melissia Dedman|   Finance|   AK|4027.0| 43| 1290|
|       1002|Rudolph Barringer|        HR|   LA|3122.0| 43| 1445|
|       1003|      Tamra Amber|  Accounts|   AK|6217.0| 47| 1291|
+-----------+-----------------+----------+-----+------+---+-----+
only showing top 4 rows



In [28]:
spark.sql("""
            SELECT 
                employee_name, age, salary,
                CASE WHEN age > 45 THEN salary + 500 else salary END AS nuevo_salario
            FROM mi_df_tabla 
          """).show(4)

+-----------------+---+------+-------------+
|    employee_name|age|salary|nuevo_salario|
+-----------------+---+------+-------------+
|        Nitz Leif| 26|6131.0|       6131.0|
|  Melissia Dedman| 43|4027.0|       4027.0|
|Rudolph Barringer| 43|3122.0|       3122.0|
|      Tamra Amber| 47|5717.0|       6217.0|
+-----------------+---+------+-------------+
only showing top 4 rows



10. Guardar en un DF los empleados cuya edad es >= 45.

In [34]:
mi_df.filter(col("age") > 45).write.csv("/mnt/d/Proyectos/Tutorial-SparkAWS/data/agrupados/punto10.csv")